In [1]:
import pandas as pd
from config import OPENAI_Key
import openai
from openai import OpenAI
import json

openai.api_key = OPENAI_Key
OPENAI_API_KEY = OPENAI_Key
client = OpenAI(api_key=OPENAI_API_KEY)
import random

In [22]:
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [ ]:
# # Helper function: get embeddings for a text
# def get_embeddings(text):
#     response = openai.Embedding.create(
#         model="text-embedding-ada-002",
#         input=text.replace("\n", " ")
#     )
#     embedding = response['data'][0]['embedding']
#     return embedding

# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# # Function to calculate cosine similarity
# def calculate_cosine_similarity(a, b):
#     return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))[0][0]

# # Generate embeddings for text and labels
# def get_embeddings(texts):
#     response = openai.Embedding.create(
#         input=texts,
#         engine="text-embedding-ada-002",  # Choose an embeddings engine
#     )
#     return np.array([item['embedding'] for item in response['data']])

# # Your text to classify and possible labels
# text_to_classify = "I want to adopt a healthier lifestyle."
# labels = ["health and fitness", "entertainment", "education", "travel"]

# # Generate embeddings
# text_embedding = get_embeddings([text_to_classify])
# label_embeddings = get_embeddings(labels)

# # Calculate similarity and classify
# similarities = [calculate_cosine_similarity(text_embedding, label_emb) for label_emb in label_embeddings]
# classification_label = labels[np.argmax(similarities)]

# print(f"Text: '{text_to_classify}' is classified as '{classification_label}'")

Could also classify conditions into specific types temporal/spacial...


In [2]:
# Define a function to replace "not numbered" with the value in "Variable Label"
def replace_not_numbered(row):
    if row["Question number\n (Questionnaire file)"] == "not numbered":
        return row["Variable Label\n (Data files)"]
    else:
        return row["Question number\n (Questionnaire file)"]

In [78]:
survey_questions  = pd.read_excel('Survey Questions Overview.xlsx', sheet_name='Wave 1', engine='openpyxl')
# Clean the survey questions dataframe to make it more usable for mapping
# Fill forward non-null ADICO Category values to apply them to all relevant rows, Specify the columns to forward fill excluding "Values" and "Value labels"
columns_to_ffill = [col for col in survey_questions.columns if col not in ["Values", "Value labels"]]

# Forward fill the specified columns
survey_questions[columns_to_ffill] = survey_questions[columns_to_ffill].ffill()


# Apply the function to replace "not numbered" with the value in "Variable Label"
survey_questions["Question number\n (Questionnaire file)"] = survey_questions.apply(replace_not_numbered, axis=1)


# Set the index to {value of "Question number\n (Questionnaire file)"} + "_" + {str(value of "Values")}
survey_questions.set_index(survey_questions["Variable Label\n (Data files)"] + "_" + survey_questions["Values"].astype(str), inplace=True)
survey_questions.drop("ID_nan", inplace=True)

question_answers_list = []
# Define a function to create the combined string
def combine_description_and_labels(group, question_answers_list, qnum):
    question_subset = survey_questions[survey_questions["Question number\n (Questionnaire file)"] == group["Question number\n (Questionnaire file)"].iloc[0]]
    # Check if it's the first row instance with the current "Question number\n (Questionnaire file)" column value
    first_instance_index = question_subset[question_subset.duplicated(subset=["Question number\n (Questionnaire file)"], keep="first")].index
    combined_string = ""
    if first_instance_index.size != 0:
        first_description = question_subset["Description"].iloc[0]
        if first_description != group["Description"].iloc[0]:
         # If not the first instance, start with the first instance's "Description" column value
            combined_string += first_description
    # Concatenate the current row's "Description" and all "Value labels" values
    combined_string += str(group["Description"].iloc[0])# + " " + "; ".join(group["Value labels"].astype(str)))
    question_answers_list = question_answers_list + [combined_string] * group.shape[0]  # Extend the list with the combined strings
    return question_answers_list

# Group by "Variable Label\n (Data files)" and apply the function to create the combined string
for group in survey_questions.groupby("Variable Label\n (Data files)",sort=False):
    question_answers_list = combine_description_and_labels(group[1], question_answers_list, group[1]["Question number\n (Questionnaire file)"].iloc[0])
survey_questions["question_answers_combined"] = question_answers_list


# Since the dataset has multiple rows per question for different value labels, we'll create a unique mapping
# Create the new mapping dictionary
question_adico_mapping = survey_questions[['question_answers_combined','ADICO Category',"Variable Label\n (Data files)"]].drop_duplicates().set_index('question_answers_combined')['ADICO Category']


# Filter out questions that are categorized as Attributes, Conditions, or Aims for clarity in analysis
attributes = [k for k, v in question_adico_mapping.items() if 'Attribute' in str(v) or 'Attribute/Condition' in str(v)]
conditions = [k for k, v in question_adico_mapping.items() if 'Condition' in str(v) or 'Aim/Condition' in str(v) or 'Attribute/Condition' in str(v)]
aims = [k for k, v in question_adico_mapping.items() if 'Aim' in str(v) or 'Aim/Condition' in str(v)]

In [79]:
attributes

['Age',
 'What gender do you identify with?',
 'What race do you self-identify as? (USA only)',
 'What ethnic group do you identify as? (Indonesia only)',
 'What is the highest level of education you have completed?',
 'What is the highest level of education you have completed?',
 'What is the highest level of education you have completed?',
 'What is the highest level of education you have completed?',
 'Employment status',
 'Employment status',
 'Employment status',
 'State of residence (USA only)',
 'Zipcode or postal code',
 'What category best describes your current home or accommodation?',
 'What category best describes your current home or accommodation?',
 '(If not mobile home) Please select all the floors you use:',
 '(If not mobile home) Please select all the floors you use:Basement',
 '(If not mobile home) Please select all the floors you use:Ground floor',
 '(If not mobile home) Please select all the floors you use:First floor or above',
 'Employer type',
 'Employer type',


In [341]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def categorizeADICO(question):
    prompt = f"Task: Categorization. Instructions: Categorize whether the survey question represents an 'Aim', a 'Condition' or 'Aim/Condition'. Survey Question: {question} Categories: - Aim: Actions performed or intended by the responder. - Condition: External factors influencing the responder's decision."

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    outputs = model.generate(input_ids, max_new_tokens=20, )
    return tokenizer.decode(outputs[0],skip_special_tokens=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [344]:
aim_quest = survey_questions[survey_questions['ADICO Category'].apply(lambda x: "Aim" in x)].head(20).copy()
# Sample questions
aim_quest['model_adico'] = aim_quest['question_answers_combined'].apply(categorizeADICO)
aim_quest

In [6]:
#For now we make a random selection of questions to identify the ADICO components but later we can provide the model with a predetermined selection
sample_questions_answers = random.sample(list(survey_questions['question_answers_combined']),5)

print("Random questions: \n" + "\n".join(sample_questions_answers)+ "\n\n")

# Constructing the prompt
prompt = "For each of the question - answer combinations below identify if they can be used to extract an Attribute, Aim, or Condition:\n\n"
prompt += "\n".join(sample_questions_answers) + "\n\n"
prompt += "\n each question - answer pair can only be assigned to one of the three options" + "\n\n"

#Explain to the GPT what their role is:
systemMessage = "You are a helpful assistant that reviews question - answer combinations from a survey and identifies if they can be used to define an Attribute, Aim, or Condition. You always output the same list of questions with their identified component(s) in JSON structure that will be read using the following: categorized_questions_json = json.loads(response.choices[0].message.content) and categorized_questions = pd.DataFrame(categorized_questions_json['questions'])."

#Give them some context information:
contentMessage = "An institutional statement refers to a structured representation of institutions using specific elements such as Attribute, Deontic, Aim, Condition, and Or Else (ADICO). These statements are used to define and understand the impacts, actions, and conditions associated with institutional rules, norms, and shared strategies within social systems.\n\n" 
contentMessage += "Attribute: Who is performing the aim action, e.g. demographics such as “Households in Indonesia” or “police officers”.\n\n" 
contentMessage += "Aim: The action performed by the entities with the listed attributes if all conditions are met, e.g. “reinforce their foundations bi-yearly” or “close the door”.\n\n"
contentMessage += "Condition: External factors that influence the demographic into performing the aim action, e.g. “if they live with more than four people” or “if the alarm goes off”\n\n"
contentMessage += "Combining these elements produces the following Institutional shared strategy: “Police officers (Attribute) lock the doors of their vehicle (Aim) if they leave the vehicle (Condition)”'\n\n"


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": systemMessage},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": contentMessage},
  ]
)

# Parse JSON
categorized_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
categorized_questions = pd.DataFrame(categorized_questions_json["questions"])

# Display dataframe
categorized_questions

Random questions: 
Please indicate if you have already implemented any of these nonstructural measures or if you intend to do so in the futureStoring or placing important possessions (such as documents or expensive furniture) in such a manner to avoid flood damage I have already implemented this non-structural measure; I intend to implement this non-structural measure in the next 6 months; I intend to implement this non-structural measure in the next 12 months; I intend to implement this non-structural measure in the next 2 years; I intend to implement this non-structural measure in future, after 2 years; I do not intend to implement this non-structural measure
Employer typeIndustry type Agriculture, Forestry, Fishing and Hunting; Mining, Oil and Gas Extraction, and Utilities; Construction; Manufacturing; Wholesale Trade; Retail Trade; Transportation and Warehousing; Media, Communications, and Digital Entertainment; Finance, Accounting, and Consulting; Insurance; Real Estate, Rental, a

RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}

1st Successful Attempt:

Random questions: 

1. During times of hardship, my household can access the financial support I need (e.g. such as access to credit at a bank) - Attribute, Condition
2. Purchasing sandbags, or other water barriers - I intend to implement this non-structural measure in the future, after 2 years - Aim, Condition
3. My household can rely on the support from my government when I need help (e.g. receiving funding or support in the event of a natural disaster) - Attribute, Condition
4. What race do you self-identify as? (USA only) - Attribute
5. Keeping a working flashlight and/or a battery-operated radio and/or emergency kit in a convenient location - I intend to implement this non-structural measure in the next 12 months - Aim, Condition

In [ ]:
# Random sample of questions for demonstration
sample_questions_answers = random.sample(list(survey_questions['question_answers_combined']), 5)

# Simplified and integrated prompt
prompt = """You are a helpful assistant reviewing question-answer combinations from a survey to identify if they define an Attribute, Aim, or Condition based on the ADICO framework. 
An Attribute refers to demographics performing the aim action, e.g., “Households in Indonesia”. 
An Aim is the action performed if conditions are met, e.g., “reinforce foundations bi-yearly”. 
A Condition influences the action, e.g., “if they live with more than four people”.

Please always output the same list of questions with their identified component(s)  in JSON structure that will be read using the following: categorized_questions_json = json.loads(response.choices[0].message.content) and categorized_questions = pd.DataFrame(categorized_questions_json['questions']).

For each of the question-answer combinations below, identify them as an Attribute, Aim, or Condition:

""" + "\n".join(sample_questions_answers)

response = client.chat.completions.create(
    model="gpt-3.5-turbo",  # Adjust engine as necessary
    messages=[
    {"role": "user", "content": prompt},
    ],
    temperature=0.5,
    max_tokens=250,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

# Parse JSON
categorized_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
categorized_questions = pd.DataFrame(categorized_questions_json["questions"])

# Display dataframe
categorized_questions

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

What about converting to ADICO Structure:

In [ ]:
# Constructing the prompt
prompt = "Generate Institutional shared strategy statements using all of the following sets of questions and answers containing Aims and Conditions:\n\n"
prompt += ",\n".join(aims_conditions) 

#Explain to the GPT what their role is:
systemMessage = "You are a helpful assistant that converts questions and answers from surveys into Institutional Behaviour Statements. You always output a list of statements that follow the following JSON structure: \n\n"
systemMessage += '{"statements":["Attribute":"Households in the Netherlands", "Aim":"do {Aim}", "Condition":"if {Condition(s)}]"”"\n\n"'
systemMessage += "Your output will be read using the following: converted_questions = json.loads(response.choices[0].message.content) and converted_questions = pd.DataFrame(converted_questions_json['statements'])."

#Give them some context information:
contentMessage = "An institutional statement refers to a structured representation of institutions using specific elements such as Attribute, Deontic, Aim, Condition, and Or Else (ADICO). These statements are used to define and understand the impacts, actions, and conditions associated with institutional rules, norms, and shared strategies within social systems.\n\n" 
contentMessage += "Attribute: Who or what the impacts of the institution apply to, e.g. “Households in Indonesia” or “police officers”.\n\n" 
contentMessage += "Aim: The definition of the impact that is applied or the action performed, e.g. “reinforce their foundations bi-yearly” or “close the door”.\n\n"
contentMessage += "Condition: the conditions that need to be satisfied in order for the aim to occur, e.g. “if they live with more than four people” or “if the alarm goes off”\n\n"
contentMessage += "Combining all these elements produces the following Institutional shared strategy: “Police officers (Attribute) lock the doors of their vehicle (Aim) if they leave the vehicle (Condition)”'\n\n"


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": systemMessage},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": contentMessage},
  ]
)

# Parse JSON
converted_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
converted_questions = pd.DataFrame(converted_questions_json["statements"])

# Display dataframe
converted_questions

,Attribute,Aim,Condition
0,Households in the Netherlands,do not intend to implement the structural meas...,if they have not personally experienced any ki...
1,Households in the Netherlands,do not intend to implement the structural meas...,if they have not implemented or intend to impl...
2,Households in the Netherlands,do not intend to implement the structural meas...,if they have not implemented or intend to impl...
3,Households in the Netherlands,do not intend to implement the structural meas...,if they have not implemented or intend to impl...
4,Households in the Netherlands,do not intend to implement the structural meas...,"if they do not expect other sources, other tha..."
5,Households in the Netherlands,do not intend to implement the structural meas...,if they have not implemented or intend to impl...
6,Households in the Netherlands,do not intend to implement the structural meas...,if they have not implemented or intend to impl...
7,Households in the Netherlands,do not intend to implement the structural meas...,if they do not have children under the age of ...
8,Households in the Netherlands,do not intend to implement the structural meas...,"if they do not expect other sources, other tha..."


In [362]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def intoIG(question):
    prompt = f"Task: Rephrashing. Instructions: Convert the following Survey Question into an descriptive phrase. Phrase Types: e.g. 'reinforce their foundations bi-yearly' or  'if they live with more than four people'. Survey Question: {question} "

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    outputs = model.generate(input_ids, max_new_tokens=100, )
    return tokenizer.decode(outputs[0],skip_special_tokens=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [363]:
aim_quest = survey_questions[survey_questions['ADICO Category'].apply(lambda x: "Aim" in x)].head(20).copy()
# Sample questions
aim_quest['IGVersion'] = aim_quest['question_answers_combined'].apply(intoIG)
aim_quest

,ADICO Category,Question number\n (Questionnaire file),Variable Label\n (Data files),Description,Values,Value labels,question_answers_combined,IGVersion
Q8_move_out_1.0,Aim/Condition,Q8,Q8_move_out,How much longer are you planning on staying in...,1.0,I am looking to move in 1 year or less,How much longer are you planning on staying in...,How long are you planning on staying in your c...
Q8_move_out_2.0,Aim/Condition,Q8,Q8_move_out,How much longer are you planning on staying in...,2.0,1-5 years,How much longer are you planning on staying in...,How long are you planning on staying in your c...
Q8_move_out_3.0,Aim/Condition,Q8,Q8_move_out,How much longer are you planning on staying in...,3.0,5-10 years,How much longer are you planning on staying in...,How long are you planning on staying in your c...
Q8_move_out_4.0,Aim/Condition,Q8,Q8_move_out,How much longer are you planning on staying in...,4.0,More than 10 years,How much longer are you planning on staying in...,How long are you planning on staying in your c...
Q8_move_out_98.0,Aim/Condition,Q8,Q8_move_out,How much longer are you planning on staying in...,98.0,Don’t know,How much longer are you planning on staying in...,How long are you planning on staying in your c...
Q9_easy_leave_1.0,Aim/Condition,Q9,Q9_easy_leave,How easy or difficult would it be to leave the...,1.0,1 - It would be difficult to leave this area,How easy or difficult would it be to leave the...,How easy or difficult would it be to leave the...
Q9_easy_leave_2.0,Aim/Condition,Q9,Q9_easy_leave,How easy or difficult would it be to leave the...,2.0,2,How easy or difficult would it be to leave the...,How easy or difficult would it be to leave the...
Q9_easy_leave_3.0,Aim/Condition,Q9,Q9_easy_leave,How easy or difficult would it be to leave the...,3.0,3,How easy or difficult would it be to leave the...,How easy or difficult would it be to leave the...
Q9_easy_leave_4.0,Aim/Condition,Q9,Q9_easy_leave,How easy or difficult would it be to leave the...,4.0,4,How easy or difficult would it be to leave the...,How easy or difficult would it be to leave the...
Q9_easy_leave_5.0,Aim/Condition,Q9,Q9_easy_leave,How easy or difficult would it be to leave the...,5.0,5 - I could leave this area very easily,How easy or difficult would it be to leave the...,How easy or difficult would it be to leave the...


In [364]:
aim_quest.at['Q8_move_out_1.0','IGVersion']

'How long are you planning on staying in your current house?'

In [ ]:
print(prompt)

Generate Institutional shared strategy statements based on the following sets of questions and answers containing Aims and Conditions:

Aim: Please indicate if you have already implemented any of these structural measures or if you intend to do so in the futureStrengthen the housing foundations to withstand water pressures Response: I do not intend to implement this structural measureand Condition: Have you ever personally experienced a flood of any kind? Response: No,
Aim: Please indicate if you have already implemented any of these structural measures or if you intend to do so in the futureStrengthen the housing foundations to withstand water pressures Response: I do not intend to implement this structural measureand Condition: Do you have children under the age of 12 or adults over the age 70 living with you? Please select all that apply (multiple answers possible)Prefer not to say Response: No,
Aim: Please indicate if you have already implemented any of these structural measures or

In [ ]:
#For now we make a random selection of questions with assigned ADICO components but later we can provide the model with a predetermined selection
selected_attributes = random.sample(attributes, 10)
selected_conditions = random.sample(conditions, 10)
selected_aims = random.sample(aims, 10)

# Constructing the prompt
prompt = "Generate Institutional shared strategy statements based on the following sets questions and answers containing Attributes, Aims and Conditions:\n\n"
prompt += "Attributes:\n" + "\n".join(selected_attributes) + "\n\n"
prompt += "Aims:\n" + "\n".join(selected_aims) + "\n\n"
prompt += "Conditions:\n" + "\n".join(selected_conditions) + "\n\n"

#Explain to the GPT what their role is:
systemMessage = "You are a helpful assistant that converts questions and answers from surveys into Institutional Behaviour Statements. You always output a list of statements that follow the following JSON structure: \n\n"
systemMessage += '{"statements":["Attribute":"People with {Attribute(s)}", "Aim":"do {Aim}", "Condition":"if {Condition(s)}]"”"\n\n"'
systemMessage += "Your output will be read using the following: converted_questions = json.loads(response.choices[0].message.content) and converted_questions = pd.DataFrame(converted_questions_json['statements'])."

#Give them some context information:
contentMessage = "An institutional statement refers to a structured representation of institutions using specific elements such as Attribute, Deontic, Aim, Condition, and Or Else (ADICO). These statements are used to define and understand the impacts, actions, and conditions associated with institutional rules, norms, and shared strategies within social systems.\n\n" 
contentMessage += "Attribute: Who or what the impacts of the institution apply to, e.g. “Households in Indonesia” or “police officers”.\n\n" 
contentMessage += "Aim: The definition of the impact that is applied or the action performed, e.g. “reinforce their foundations bi-yearly” or “close the door”.\n\n"
contentMessage += "Condition: the conditions that need to be satisfied in order for the aim to occur, e.g. “if they live with more than four people” or “if the alarm goes off”\n\n"
contentMessage += "Combining all these elements produces the following Institutional shared strategy: “Police officers (Attribute) lock the doors of their vehicle (Aim) if they leave the vehicle (Condition)”'\n\n"


response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": systemMessage},
    {"role": "user", "content": prompt},
    {"role": "assistant", "content": contentMessage},
  ]
)

# Parse JSON
converted_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
converted_questions = pd.DataFrame(converted_questions_json["statements"])

# Display dataframe
converted_questions

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
# Parse JSON
categorized_questions_json = json.loads(response.choices[0].message.content)

# Convert JSON to dataframe
categorized_questions = pd.DataFrame(categorized_questions_json["statements"])

# Display dataframe
categorized_questions

,Attribute,Aim,Condition
0,People with industry type as Retail,buy a spare power generator to power their home,if they intend to implement this non-structura...
1,People with highest level of education as Leve...,buy a spare power generator to power their home,if they intend to implement this non-structura...
2,"People with industry type as Public Sector, Go...",store or place important possessions in a mann...,if they intend to implement this non-structura...
3,People with highest level of education as High...,store or place important possessions in a mann...,if they intend to implement this non-structura...
4,People with industry type as Government and no...,raise the level of the ground floor above the ...,if they intend to implement this structural me...
5,People with highest level of education as Post...,raise the level of the ground floor above the ...,if they intend to implement this structural me...
6,People with industry type as Financial Service...,store emergency food and water supplies,if they have already implemented this non-stru...
7,People with highest level of education as Coll...,store emergency food and water supplies,if they have already implemented this non-stru...
8,"People with industry type as Engineering, Comp...",ask/petition government representatives to inc...,if they intend to implement this non-structura...


FIRST ATTEMPT:
'Institutional Behaviour Statements:\n\n
1. Retired individuals employed by private, for-profit companies in the natural resources industry (Attributes) intend to raise the level of the ground floor above the most likely flood level (Aim) if they plan to stay in their current house for 1-5 years (Condition).\n\n
2. Individuals with less than secondary education completed, working for other employers in the health or social care industry (Attributes) intend to strengthen their housing foundations to withstand water pressures (Aim) if they have the ability to undertake the structural measure in the next 6 months (Condition).\n\n
3. Individuals aged 35-44 working in the manufacturing industry of automobiles or automobile-related fields (Attributes) intend to install anti-backflow valves on pipes (Aim) if they plan to stay in their current residence for 1-5 years (Condition).\n\n
4. Individuals in the high school education category working in the retail industry (Attributes) intend to be active members of a community group aimed at making the community safer (Aim) if they have already implemented non-structural measures for emergency preparedness (Condition).'


SECOND:

1. People with a Villa accommodation category intend to install anti-backflow valves on pipes in the future, after 2 years, if they very frequently engage with social media. \n\n
2. People in the Construction industry intend to coordinate with neighbors in case of floods if not at home, within the next 6 months if more than five households have taken some adaptive action towards flooding.\n\n
3. Individuals with a 4-year university degree store emergency food and water supplies if they find it extremely ineffective to store them, and they are unable to store or place important possessions to avoid flood damage.\n\n
4. Students aim to implement the strategy of keeping a working flashlight and/or a battery-operated radio and/or emergency kit in a convenient location within the next 2 years if they do not trust social media at all.\n\n
5. Self-employed individuals engage with social media very frequently to take adaptive actions towards flooding if they intend to install a pump and/or drainage system to drain floodwater, finding it extremely effective.\n\n
6. People involved in heavy machinery or manufacturing industry types do not intend to implement the strategy of fixing water barriers if they are part-time workers (less than 8 hours a week) and find it extremely ineffective to store emergency food and water supplies.\n\n
7. Individuals in the real estate industry aim to store emergency food and water supplies within the next 6 months if they find it extremely ineffective and do not trust social media at all.\n\n
8. People with a university higher degree (Masters, MBA, PhD) do not trust social media at all and store emergency food and water supplies if they are unable to store or place important possessions to avoid flood damage.\n\n
9. Those involved in transport retail or wholesale industry types intend to move or store valuable assets on higher floors or elevated areas if it is not easy to leave their current place of residence, and they do not trust social media at all.\n\n
10. People working part-time (less than 8 hours a week) aim to implement the strategy of installing anti-backflow valves on pipes after 2 years if they do not find it easy to leave their current place of residence.'

In [ ]:
selected_aims

['Raising the level of the ground floor above the most likely flood levelInstalling a pump and/or one or more system(s) to drain flood water I have already implemented this structural measure; I intend to implement this structural measure in the next 6 months; I intend to implement this structural measure in the next 12 months; I intend to implement this structural measure in the next 2 years; I intend to implement this structural measure in the future, after 2 years; I do not intend to implement this structural measure',
 'Keeping a working flashlight and/or a battery-operated radio and/or emergency kit in a convenient locationStoring emergency food and water supplies I have already implemented this non-structural measure; I intend to implement this non-structural measure in the next 6 months; I intend to implement this non-structural measure in the next 12 months; I intend to implement this non-structural measure in the next 2 years; I intend to implement this non-structural measure 